1. Dataset has been downloaded and saved

2. Import required Libraries

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

Matplotlib is building the font cache; this may take a moment.


3. Read dataset and do preprocessing

In [3]:
df = pd.read_csv(
    '../Dataset/spam.csv',
    delimiter = ',',
    encoding = 'latin-1'
)

In [4]:
df.drop(
    ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],
    axis = 1,
    inplace = True
)

In [5]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1, 1)

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size = 0.15
)

In [7]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences, maxlen = max_len)

5. Add Layers (LSTM, Dense-(Hidden Layers), Output)

In [8]:
def RNN():
    inputs = Input(name = 'inputs', shape = [max_len])
    layer = Embedding(max_words, 50, input_length = max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name = 'FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name = 'out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs = inputs, outputs = layer)
    return model

4. Create Model

In [9]:
model = RNN()

6. Compile the model

In [10]:
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(), metrics = ['accuracy'])

7. Fit the model

In [11]:
model.fit(
    sequences_matrix,
    Y_train,
    batch_size = 128,
    epochs=10,
    validation_split = 0.2,
    callbacks=[EarlyStopping(monitor = 'val_loss', min_delta = 0.0001)])

Epoch 1/10
30/30 [==============================] - 12s 266ms/step - loss: 0.3099 - accuracy: 0.8846 - val_loss: 0.1912 - val_accuracy: 0.9768
Epoch 2/10
30/30 [==============================] - 7s 238ms/step - loss: 0.0826 - accuracy: 0.9813 - val_loss: 0.0572 - val_accuracy: 0.9831


8. Save the model

In [12]:
model.save('./spam.h5')

9. Test the model

In [13]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences, maxlen = max_len)

In [14]:
accr = model.evaluate(test_sequences_matrix, Y_test)

27/27 [==============================] - 1s 37ms/step - loss: 0.0480 - accuracy: 0.9844


In [15]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.048
  Accuracy: 0.984
